In [1]:
# Importar bibliotecas necessárias
from flask import Flask, request, jsonify
from werkzeug.serving import make_server
import threading
import requests
import time

# Definindo a aplicação Flask
app = Flask(__name__)

# Endpoint para somar dois números
@app.route("/soma", methods=["POST"])
def somar():
    dados = request.get_json()  # Obter os dados JSON da requisição
    numero1 = dados.get("numero1", 0)
    numero2 = dados.get("numero2", 0)
    resultado = numero1 + numero2
    return jsonify({"resultado": resultado})  # Retornar o resultado em formato JSON

# Classe para rodar o servidor Flask em uma thread separada
class ThreadedServer(threading.Thread):
    def __init__(self, app):
        threading.Thread.__init__(self)
        self.srv = make_server("127.0.0.1", 5000, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        print("Servidor Flask iniciado...")
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

# Iniciar o servidor Flask em uma thread separada
servidor = ThreadedServer(app)
servidor.start()

# Aguardar um pouco para garantir que o servidor esteja rodando
time.sleep(2)

# Enviar uma requisição de teste para o endpoint "/soma"
dados = {"numero1": 5, "numero2": 7}
resposta = requests.post("http://127.0.0.1:5000/soma", json=dados)

# Exibir o resultado
if resposta.status_code == 200:
    print("Resultado da soma:", resposta.json()["resultado"])
else:
    print("Erro na requisição:", resposta.status_code)

# Encerrar o servidor ao final
servidor.shutdown()


Servidor Flask iniciado...


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 18:17:28] "POST /soma HTTP/1.1" 200 -


Resultado da soma: 12


In [5]:
# Instalar bibliotecas necessárias:
# !pip install fastapi uvicorn requests

# Importar bibliotecas necessárias
from fastapi import FastAPI
from pydantic import BaseModel
from threading import Thread
import requests
import time
import uvicorn

# Definindo a aplicação FastAPI
app = FastAPI()

# Classe para validação dos dados
class Numeros(BaseModel):
    numero1: float
    numero2: float

# Endpoint para multiplicar dois números
@app.post("/multiplicacao")
def multiplicar(numeros: Numeros):
    resultado = numeros.numero1 * numeros.numero2
    return {"resultado": resultado}

# Função para rodar o servidor FastAPI em segundo plano
def iniciar_servidor():
    config = uvicorn.Config(app, host="127.0.0.1", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

# Iniciar o servidor FastAPI em uma thread separada
thread = Thread(target=iniciar_servidor)
thread.start()

# Aguardar um pouco para garantir que o servidor esteja rodando
time.sleep(2)

# Enviar uma requisição de teste para o endpoint '/multiplicacao'
dados = {"numero1": 4, "numero2": 2}
resposta = requests.post("http://127.0.0.1:8000/multiplicacao", json=dados)

# Exibir o resultado
if resposta.status_code == 200:
    print("Resultado da multiplicação:", resposta.json()["resultado"])
else:
    print("Erro na requisição:", resposta.status_code)


INFO:     Started server process [246]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     127.0.0.1:41508 - "POST /multiplicacao HTTP/1.1" 200 OK
Resultado da multiplicação: 8.0
